**Working with unstructured .txt file**

**As unstructured file will only have "value" column. So first thing is to meke it structured and then we can apply transformation**

**Requirement: Find top 3 referrers**

In [0]:
#reading the file

txt_df = spark.read\
              .format('text')\
              .load("/Volumes/dev/multi_datasets/spark_data/apache-logs.txt")

display(txt_df.limit(3))

value
"83.149.9.216 - - [17/May/2015:10:05:03 +0000] ""GET /presentations/logstash-monitorama-2013/images/kibana-search.png HTTP/1.1"" 200 203023 ""http://semicomplete.com/presentations/logstash-monitorama-2013/"" ""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"""
"83.149.9.216 - - [17/May/2015:10:05:43 +0000] ""GET /presentations/logstash-monitorama-2013/images/kibana-dashboard3.png HTTP/1.1"" 200 171717 ""http://semicomplete.com/presentations/logstash-monitorama-2013/"" ""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"""
"83.149.9.216 - - [17/May/2015:10:05:47 +0000] ""GET /presentations/logstash-monitorama-2013/plugin/highlight/highlight.js HTTP/1.1"" 200 26185 ""http://semicomplete.com/presentations/logstash-monitorama-2013/"" ""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"""


In [0]:
# Create a regex expression to seperate the content of txt file.

txt_reg = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\S+) "(\S+)" "([^"]*)'

**Spark gives us regexp_extract() function to extract one field from a regular expression**

**regexp_extract takes 3- argument**
**1st- column name which need to be applied regex**
**2nd-regular expression**
**3rd-position after splitting the value using reg exp**

In [0]:
from pyspark.sql.functions import regexp_extract
log_df = txt_df.select(regexp_extract('value', txt_reg, 1).alias('ip'),
                       regexp_extract('value', txt_reg, 4).alias('date'),
                       regexp_extract('value', txt_reg, 6).alias('image'),
                       regexp_extract('value', txt_reg, 10).alias('referrer'))
display(log_df.limit(3))

ip,date,image,referrer
83.149.9.216,17/May/2015:10:05:03 +0000,/presentations/logstash-monitorama-2013/images/kibana-search.png,http://semicomplete.com/presentations/logstash-monitorama-2013/
83.149.9.216,17/May/2015:10:05:43 +0000,/presentations/logstash-monitorama-2013/images/kibana-dashboard3.png,http://semicomplete.com/presentations/logstash-monitorama-2013/
83.149.9.216,17/May/2015:10:05:47 +0000,/presentations/logstash-monitorama-2013/plugin/highlight/highlight.js,http://semicomplete.com/presentations/logstash-monitorama-2013/


In [0]:
referrer_df = log_df.groupBy("referrer")\
                    .count()\
                    .orderBy("count", ascending=False)
display(referrer_df.limit(5))

referrer,count
-,4073
http://semicomplete.com/presentations/logstash-puppetconf-2012/,689
http://www.semicomplete.com/projects/xdotool/,656
http://semicomplete.com/presentations/logstash-scale11x/,406
http://www.semicomplete.com/articles/dynamic-dns-with-dhcp/,335


**But here is the problem as we can see same website not being aggregated together. Also we can see nulls and hyphen in the referrer field**

**trim(referrer) removes leading/trailing spaces || != '-' removes rows where:referrer is just - (common placeholder in logs for missing values)**

**substring_index("referrer", "/", 3) -->Splits the string using / and Keeps first 3 parts**

**http:(/-1st part)(/-2nd part)semicomplete.com(/-3rd part)presentations(/-4th part)logstash-puppetconf-2012/**




In [0]:
from pyspark.sql.functions import substring_index
referrer_df = log_df.where("trim(referrer) != '-'")\
                    .withColumn("referrer", substring_index("referrer", "/", 3))\
                    .groupBy("referrer")\
                    .count()\
                    .orderBy("count", ascending=False)
display(referrer_df.limit(5))


referrer,count
http://www.semicomplete.com,3038
http://semicomplete.com,2001
http://www.google.com,123
https://www.google.com,105
http://stackoverflow.com,34


# **Congratulation we have top 3 referrers now**